# Using ANN to predict gz

Put together by Thomas Martin, thomasmartin@mines.edu, all errors are mine

If you are interested in graident boosting, here is a good place to start: https://xgboost.readthedocs.io/en/latest/tutorials/model.html

This is a supervised machine learning method.

In [ ]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)


In [ ]:
import tensorflow as tf
from tensorflow import keras
print('tf version:', tf.__version__)
print('keras version:', keras.__version__)

tf version: 2.3.0
keras version: 2.4.0


In [ ]:
# If you have installation questions, please reach out
import pandas as pd # data storage
import numpy as np  # math and stuff

import sklearn  

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import matplotlib.pyplot as plt # plotting utility

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/Merged_dataset.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,21,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num', 'sample' ], axis=1)

In [ ]:
df = df[df.gz_pchip_interp >= 0]

In [ ]:
df

,depth_ft,USGS_ID,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
20910,7741.0,F028,8.708905,38.611032,NaN,-27.054580,2.514808,2.838095,26.226486,0.171561,0.084543,38.090966,4.0313,4.0320,NaN,NaN,NaN,NaN,NaN,14.5,5.55,1.277,0.0263,0.6987,0.0116,2.8101,0.0112,33.9921,0.0332,0.0826,0.0021,0.1116,0.0011,0.9947,0.0045,4.564,0.0073,0.1641,0.0013,0.0085,0.0001,0.0186,0.0008,1.4531,0.0029,0.0011,0.0,0.0044,0.0001,0.0,0.0008,0.0023,0.0008,0.0663,0.0049
20911,7741.1,F028,8.706406,38.480560,NaN,-26.947395,2.513834,2.821543,26.074844,0.172288,0.082785,38.233341,3.9834,3.9834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20912,7741.2,F028,8.703908,38.361977,NaN,-26.833857,2.512615,2.843588,25.922649,0.173380,0.081614,38.393699,3.9837,3.9836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20913,7741.3,F028,8.701412,38.197782,NaN,-26.728386,2.511544,2.812542,25.772518,0.174471,0.087358,38.576009,3.9840,3.9840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20914,7741.4,F028,8.698913,38.056724,NaN,-26.648379,2.510388,2.811718,25.621972,0.175562,0.091667,38.777806,3.9843,3.9843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210386,13378.0,PDRMT,6.057740,60.192220,71.87150,-40.570650,2.603100,6.168050,8.262480,0.167140,0.047800,58.623425,4.3717,4.3658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210387,13378.1,PDRMT,6.058370,60.405740,72.21565,-40.313090,2.607350,6.127460,8.180290,0.173980,0.045200,58.915458,6.6922,6.7092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210388,13378.2,PDRMT,6.059000,60.512750,72.51256,-40.044420,2.611640,6.081560,8.098920,0.177590,0.042590,59.132423,6.7190,6.7313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210389,13378.3,PDRMT,6.059630,60.585970,72.60000,-39.765300,2.616020,6.036370,8.018350,0.177050,0.039990,59.282241,4.3369,4.3273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Loading in Dataset

In [ ]:
df.columns.values

array(['depth_ft', 'USGS_ID', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 'GR_smooth', 'gz_linear_interp',
       'gz_pchip_interp', 'perm_air_md', 'perm_klink_md', 'He_por',
       'horz_perm_md', 'effective_por', 'por_percent',
       'stress_swanson_perm_md', 'Na', 'Na Err', 'Mg', 'Mg Err', 'Al',
       'Al Err', 'Si', 'Si Err', 'P', 'P Err', 'S', 'S Err', 'K', 'K Err',
       'Ca', 'Ca Err', 'Ti', 'Ti Err', 'Cr', 'Cr Err', 'Mn', 'Mn Err',
       'Fe', 'Fe Err', 'Co', 'Co Err', 'Ni', 'Ni Err', 'Cu', 'Cu Err',
       'Zn', 'Zn Err', 'Ba', 'Ba Err'], dtype=object)

In [ ]:
dataset = df[[
  'depth_ft', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
  'RESD', 'PHIN', 'PHID', 'GR_smooth', 'gz_pchip_interp'
    ]]

In the next code block, we will remove the rows without data, and change string NaN's to np.nans

In [ ]:
dataset.replace('NaN',np.nan, regex=True, inplace=True)# 
dataset = dataset.dropna()
np.shape(dataset)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


(7760, 12)

In [ ]:
dataset.head(3)

,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,GR_smooth,gz_pchip_interp
74613,10263.0,7.791549,9.776502,65.457289,14.813261,2.473177,2.319021,61.975083,0.061525,0.107073,12.573542,3.9269
74614,10263.1,7.787731,9.690210,65.503018,14.683679,2.471589,2.319280,62.200601,0.061131,0.108135,12.232586,3.9448
74615,10263.2,7.784906,9.690018,65.548747,14.556871,2.469616,2.320136,62.423503,0.060738,0.109207,11.994466,3.9576


In [ ]:
logs = ['depth_ft', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 'GR_smooth']

xrf  = [ 'Na', 'Mg', 'Al',  'Si',  'P',  'S',  'K', 'Ca',  'Ti',  'Cr',  'Mn',
       'Fe',  'Co',  'Ni',  'Cu', 'Zn',  'Ba', ]

X = dataset[['depth_ft', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 'GR_smooth']]

Y = dataset[['gz_pchip_interp']]

In [ ]:
Y_array = np.array(Y.values)
# Y_array

## Starting to set up the ML model params

In [ ]:
seed = 7 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y_array, test_size=test_size)

In [ ]:
scaler = MinMaxScaler()
# Train
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
# test
X_test_scaled = scaler.fit_transform(X_test.astype(np.float64))

y_train_scaled = scaler.fit_transform(y_train.astype(np.float64))

y_test_scaled = scaler.fit_transform(y_test.astype(np.float64))

In [ ]:
svm_reg_rbf = svm.SVR(kernel='rbf', C=2, max_iter=1000, tol=0.0005, gamma= 0.05)

svm_reg_rbf.fit(X_train, y_train)
preds = svm_reg_rbf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.846443


# Polynomial Kernal

In [ ]:
parameters = {
    'C': [ 1, 2, 3, 4],
    'tol':[0.0005,0.001, 0.003, 0.01],
    'gamma': [ 0.01, 0.05, 0.1, 0.2],
    'max_iter': [100, 200, 500, 1000],
    'degree': [2,3,4,5]
}

estimator = svm.SVR(kernel='poly')

In [ ]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    n_jobs = 8,
    cv = 5,
    verbose = True
)

In [ ]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1024 candidates, totalling 5120 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    5.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   25.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   57.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:  2.8min
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:  4.0min
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:  5.5min
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:  7.1min
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:  9.0min
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed: 11.2min
[Parallel(n_jobs=8)]: Done 5120 out of 5120 | elapsed: 11.5min finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = col

GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='poly',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=8,
             param_grid={'C': [1, 2, 3, 4], 'degree': [2, 3, 4, 5],
                         'gamma': [0.01, 0.05, 0.1, 0.2],
                         'max_iter': [100, 200, 500, 1000],
                         'tol': [0.0005, 0.001, 0.003, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [ ]:
grid_search.best_estimator_

SVR(C=2, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma=0.01,
    kernel='poly', max_iter=100, shrinking=True, tol=0.0005, verbose=False)

In [ ]:
svm_reg_poly = svm.SVR(kernel='poly', degree=2, C=2, max_iter=100, tol=0.0005, gamma= 0.01)

svm_reg_poly.fit(X_train, y_train)
preds = svm_reg_poly.predict(X_test)



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [ ]:
rmse2 = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse2))

RMSE: 503542335.130477


# Sigmoid Kernal

In [ ]:
parameters_sig = {
    'C': [ 1, 2, 3, 4],
    'tol':[0.0005,0.001, 0.003, 0.01],
    'gamma': [ 0.01, 0.05, 0.1, 0.2],
    'max_iter': [100, 200, 500, 1000],
    'degree': [2,3,4]
}

estimator = svm.SVR(kernel='sigmoid')

In [ ]:
grid_search_sig = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_sig,
    n_jobs = 8,
    cv = 5,
    verbose = True
)

In [ ]:
grid_search_sig.fit(X_train, y_train)

Fitting 5 folds for each of 768 candidates, totalling 3840 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    5.0s
[Parallel(n_jobs=8)]: Done 216 tasks      | elapsed:   30.4s
[Parallel(n_jobs=8)]: Done 466 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done 816 tasks      | elapsed:  2.1min
[Parallel(n_jobs=8)]: Done 1266 tasks      | elapsed:  3.3min
[Parallel(n_jobs=8)]: Done 1816 tasks      | elapsed:  4.7min
[Parallel(n_jobs=8)]: Done 2466 tasks      | elapsed:  6.5min
[Parallel(n_jobs=8)]: Done 3216 tasks      | elapsed:  8.4min
[Parallel(n_jobs=8)]: Done 3840 out of 3840 | elapsed: 10.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminat

GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='sigmoid',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=8,
             param_grid={'C': [1, 2, 3, 4], 'degree': [2, 3, 4],
                         'gamma': [0.01, 0.05, 0.1, 0.2],
                         'max_iter': [100, 200, 500, 1000],
                         'tol': [0.0005, 0.001, 0.003, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [ ]:
grid_search.best_estimator_

SVR(C=2, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma=0.01,
    kernel='poly', max_iter=100, shrinking=True, tol=0.0005, verbose=False)

In [ ]:
svm_reg_sig = svm.SVR(kernel='sigmoid', degree=2, C=2, max_iter=100, tol=0.0005, gamma= 0.01)

svm_reg_sig.fit(X_train, y_train)
preds = svm_reg_sig.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [ ]:
rmse_sig = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse_sig))

RMSE: 1.762785
